<a href="https://colab.research.google.com/github/Dataniel31/DisasterTweetClassifier/blob/main/DisasterTweetClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disaster Tweet Classifier

## Introducción
Este proyecto tiene como objetivo clasificar tweets relacionados con desastres naturales utilizando técnicas de procesamiento de lenguaje natural (NLP) y modelos de machine learning.

## Descarga de Datasets
En esta sección, descargaremos los datasets que se utilizarán para entrenar y evaluar nuestro modelo. Asegúrate de que tienes acceso a Internet en tu entorno de Google Colab.


In [ ]:
import pandas as pd

#Descargar datasets a utilizar
!wget 'https://raw.githubusercontent.com/Dataniel31/DisasterTweetClassifier/refs/heads/main/test.csv'
!wget 'https://raw.githubusercontent.com/Dataniel31/DisasterTweetClassifier/refs/heads/main/train.csv'

# Cargar el dataset
train_df = pd.read_csv('train.csv')

# Ver las primeras filas del dataset
train_df.head()


--2024-10-20 21:35:16--  https://raw.githubusercontent.com/Dataniel31/DisasterTweetClassifier/refs/heads/main/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 420783 (411K) [text/plain]
Saving to: ‘test.csv.1’

test.csv.1          100%[===================>] 410.92K  --.-KB/s    in 0.05s   

2024-10-20 21:35:16 (8.51 MB/s) - ‘test.csv.1’ saved [420783/420783]

--2024-10-20 21:35:16--  https://raw.githubusercontent.com/Dataniel31/DisasterTweetClassifier/refs/heads/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Exploración de Datos

En esta sección, examinaremos los datos para identificar cualquier valor faltante en las columnas y también para ver la distribución de la variable objetivo. Esto nos ayudará a entender mejor la calidad de los datos y cómo están distribuidas las clases.

### Valores Faltantes

Contaremos los valores faltantes en cada columna del conjunto de entrenamiento.

### Distribución de la Variable Objetivo

Ahora, analizaremos la distribución de la variable objetivo (`target`) para entender cómo se distribuyen las clases en el conjunto de datos.


In [ ]:
# Contar valores faltantes en cada columna
missing_values = train_df.isnull().sum()
print("Valores faltantes en cada columna:\n", missing_values)

# Ver la distribución de la columna target
target_distribution = train_df['target'].value_counts()
print("\nDistribución de la variable objetivo (target):\n", target_distribution)


Valores faltantes en cada columna:
 id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

Distribución de la variable objetivo (target):
 target
0    4342
1    3271
Name: count, dtype: int64


## Limpieza de Datos

En esta sección, abordaremos los valores faltantes en el conjunto de datos. Es importante limpiar los datos para asegurar que nuestro modelo de machine learning funcione de manera efectiva.

### Manejo de Valores Faltantes

1. **Keyword**: Dado que esta columna tiene algunos valores faltantes, decidimos rellenarlos con la palabra "no_keyword". Esto nos permitirá mantener la coherencia en el conjunto de datos sin perder información valiosa.
  
2. **Location**: Esta columna presenta una gran cantidad de valores faltantes. Es importante explorar si la información de ubicación aporta algo significativo a nuestro análisis. Si determinamos que no es relevante, podríamos considerar descartarla en nuestro análisis inicial.


In [ ]:
# Rellenar valores faltantes en 'keyword' sin advertencia
train_df = train_df.copy()  # Crear una copia del df original
train_df['keyword'] = train_df['keyword'].fillna('no_keyword')

# Valores faltantes en 'keyword' han sido rellenados
missing_values_after = train_df['keyword'].isnull().sum()
print("Valores faltantes en 'keyword' después de rellenar:", missing_values_after)


Valores faltantes en 'keyword' después de rellenar: 0


## Preprocesamiento del Texto

Para preparar el texto para el modelo, realizaremos las siguientes transformaciones:

1. **Convertir todo el texto a minúsculas** para garantizar la uniformidad.
2. **Eliminar signos de puntuación y caracteres especiales** que no aportan significado al análisis.
3. **Remover URLs y menciones de usuarios** para limpiar el contenido.
4. **Eliminar stopwords** (palabras comunes que no aportan significado, como "the", "and", etc.).

A continuación, implementamos la función de limpieza y aplicamos estas transformaciones al conjunto de datos.


In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

nltk.download('stopwords')
nltk.download('punkt')

# Stopwords en inglés
stop_words = set(stopwords.words('english'))

# Limpiar el texto
def clean_text(text):
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Eliminar menciones de usuarios (@usuario)
    text = re.sub(r'@\w+', '', text)
    # Eliminar signos de puntuación y caracteres especiales
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenizar el texto
    words = word_tokenize(text)
    # Eliminar stopwords
    filtered_words = [word for word in words if word not in stop_words]
    # Unir las palabras filtradas de nuevo en un solo string
    return ' '.join(filtered_words)

# Aplicar limpieza al dataset
train_df['clean_text'] = train_df['text'].apply(clean_text)

# Ver las primeras filas para confirmar los cambios
train_df[['text', 'clean_text']].head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text,clean_text
0,Our Deeds are the Reason of this #earthquake M...,deeds reason earthquake may allah forgive us
1,Forest fire near La Ronge Sask. Canada,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,residents asked shelter place notified officer...
3,"13,000 people receive #wildfires evacuation or...",people receive wildfires evacuation orders cal...
4,Just got sent this photo from Ruby #Alaska as ...,got sent photo ruby alaska smoke wildfires pou...


## Vectorización del Texto

Ahora que tenemos el texto limpio, el siguiente paso es convertirlo en una representación numérica que el modelo de machine learning pueda entender. Para ello, utilizaremos **TF-IDF** (Term Frequency-Inverse Document Frequency) para vectorizar el texto.

Implementamos el vectorizador TF-IDF, limitando a las 5000 características más relevantes. Luego, ajustamos y transformamos el texto limpio para crear nuestra matriz de características.

A continuación, obtenemos las etiquetas del conjunto de datos.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear un vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Limitar a las 5000 características más relevantes

# Ajustar y transformar el texto limpio
X_train = tfidf_vectorizer.fit_transform(train_df['clean_text']).toarray()

# Obtener las etiquetas
y_train = train_df['target'].values

# Ver el tamaño de los datos de entrenamiento
print("Tamaño de X_train:", X_train.shape)
print("Tamaño de y_train:", y_train.shape)


Tamaño de X_train: (7613, 5000)
Tamaño de y_train: (7613,)


## División del Conjunto de Datos

En esta etapa, dividimos el conjunto de datos en dos partes: un conjunto de entrenamiento y un conjunto de validación. Esto es crucial para evaluar el rendimiento del modelo de manera efectiva y asegurarnos de que no está sobreajustado.

Utilizamos la función `train_test_split` de `sklearn` para realizar esta división, asignando el 20% de los datos al conjunto de validación. Además, aplicamos `stratify` en la variable objetivo (`y_train`) para mantener la misma proporción de clases en ambos conjuntos.

A continuación, verificamos el tamaño de los nuevos conjuntos:


In [ ]:
from sklearn.model_selection import train_test_split

# Dividir el conjunto de datos en entrenamiento y validación
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Verificar el tamaño de los nuevos conjuntos
print("Tamaño de X_train_split:", X_train_split.shape)
print("Tamaño de y_train_split:", y_train_split.shape)
print("Tamaño de X_val_split:", X_val_split.shape)
print("Tamaño de y_val_split:", y_val_split.shape)


Tamaño de X_train_split: (6090, 5000)
Tamaño de y_train_split: (6090,)
Tamaño de X_val_split: (1523, 5000)
Tamaño de y_val_split: (1523,)


## Entrenamiento del Modelo

En esta etapa, creamos un modelo de regresión logística utilizando `LogisticRegression` de la biblioteca `sklearn`. La regresión logística es un algoritmo de clasificación que se utiliza comúnmente para problemas de clasificación binaria.

### Pasos realizados:

1. **Creación del Modelo**: Definimos el modelo con un número máximo de iteraciones de 1000 y establecemos una semilla aleatoria para garantizar la reproducibilidad.
  
2. **Entrenamiento del Modelo**: Ajustamos el modelo utilizando el conjunto de entrenamiento (`X_train_split` y `y_train_split`).

3. **Predicciones**: Realizamos predicciones en el conjunto de validación (`X_val_split`).

4. **Evaluación del Modelo**: Evaluamos el rendimiento del modelo mediante un reporte de clasificación y el cálculo del F1 Score, que es una medida importante para problemas de clasificación desequilibrados.

A continuación, se muestra el código utilizado para esta sección:


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

# Crear el modelo de regresión logística
model = LogisticRegression(max_iter=1000, random_state=42)

# Entrenar el modelo
model.fit(X_train_split, y_train_split)

# Realizar predicciones en el conjunto de validación
y_val_pred = model.predict(X_val_split)

# Evaluar el modelo
print("Reporte de Clasificación:\n", classification_report(y_val_split, y_val_pred))
print("F1 Score:", f1_score(y_val_split, y_val_pred))


Reporte de Clasificación:
               precision    recall  f1-score   support

           0       0.81      0.91      0.86       869
           1       0.86      0.71      0.78       654

    accuracy                           0.82      1523
   macro avg       0.83      0.81      0.82      1523
weighted avg       0.83      0.82      0.82      1523

F1 Score: 0.7761944677284157


## Ajuste de Hiperparámetros

El ajuste de hiperparámetros es un proceso crucial que busca encontrar la mejor combinación de parámetros para optimizar el rendimiento del modelo. En esta sección, utilizamos `GridSearchCV` de la biblioteca `sklearn` para realizar una búsqueda exhaustiva de los hiperparámetros para nuestro modelo de regresión logística.

### Pasos realizados:

1. **Definición del Modelo**: Creamos una instancia del modelo de regresión logística, especificando un número máximo de iteraciones de 1000.

2. **Definición de Hiperparámetros**: Especificamos una cuadrícula de hiperparámetros para ajustar, que incluye:
   - `C`: Valores que representan el inverso de la regularización.
   - `solver`: Diferentes algoritmos para optimización (se utilizaron 'lbfgs' y 'liblinear').
   - `penalty`: Tipo de regularización (se utilizó L2).

3. **Grid Search**: Utilizamos `GridSearchCV` para realizar una búsqueda en la cuadrícula, evaluando el modelo con validación cruzada de 5 pliegues y utilizando el F1 Score como métrica de rendimiento.

4. **Resultados**: Imprimimos los mejores hiperparámetros encontrados y el mejor F1 Score.

A continuación, se muestra el código utilizado para esta sección:


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Definir el modelo
log_reg = LogisticRegression(max_iter=1000)

# Definir los hiperparámetros a ajustar
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear'],
    'penalty': ['l2']  # L2
}

# Realizar Grid Search
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid,
                           scoring='f1', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train_split, y_train_split)

# Resultados del modelo
print("Mejores Hiperparámetros:", grid_search.best_params_)
print("Mejor F1 Score:", grid_search.best_score_)


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Mejores Hiperparámetros: {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
Mejor F1 Score: 0.7321270733307358


## Evaluación del Modelo Ajustado

Después de realizar el ajuste de hiperparámetros mediante `GridSearchCV`, procedemos a evaluar el rendimiento del mejor modelo en el conjunto de validación. Esta evaluación es fundamental para entender cómo se comporta el modelo con datos que no ha visto durante el entrenamiento.

### Pasos realizados:

1. **Selección del Mejor Modelo**: Utilizamos el mejor estimador encontrado durante el ajuste de hiperparámetros para hacer predicciones en el conjunto de validación.

2. **Predicciones**: Realizamos las predicciones utilizando el conjunto de validación.

3. **Reporte de Clasificación**: Generamos un reporte de clasificación que incluye métricas como precisión, recall y F1 Score para ambas clases.

4. **Cálculo del F1 Score**: Calculamos el F1 Score para tener una medida única que combine precisión y recall, proporcionando una visión más clara del rendimiento del modelo.

A continuación, se muestra el código utilizado para esta sección:


In [ ]:
# Evaluar el modelo ajustado en el conjunto de validación
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val_split)

# Reporte de clasificación
from sklearn.metrics import classification_report

print("Reporte de Clasificación:")
print(classification_report(y_val_split, y_val_pred))

# Calcular el F1 Score
from sklearn.metrics import f1_score
f1 = f1_score(y_val_split, y_val_pred)
print("F1 Score en el conjunto de validación:", f1)


Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.81      0.91      0.86       869
           1       0.86      0.71      0.78       654

    accuracy                           0.82      1523
   macro avg       0.83      0.81      0.82      1523
weighted avg       0.83      0.82      0.82      1523

F1 Score en el conjunto de validación: 0.7761944677284157


## Predicción en el Conjunto de Test

Una vez que hemos entrenado y ajustado nuestro modelo, el siguiente paso es realizar predicciones sobre el conjunto de test. A continuación, se detallan los pasos que seguimos para llevar a cabo esta tarea:

### Pasos Realizados:

1. **Carga del Conjunto de Test**: Cargamos el conjunto de datos de prueba para realizar las predicciones.

2. **Limpieza del Texto**: Aplicamos la misma función de limpieza que utilizamos para el conjunto de entrenamiento, asegurándonos de que el texto esté en el formato adecuado para el análisis.

3. **Vectorización del Texto**: Transformamos el texto limpio en una representación numérica utilizando el mismo vectorizador TF-IDF que se utilizó para el conjunto de entrenamiento.

4. **Predicciones**: Utilizamos el modelo entrenado para realizar predicciones sobre el conjunto de test.

5. **Preparación del Archivo de Envío**: Creamos un DataFrame con los IDs y las predicciones, y guardamos este DataFrame como un archivo CSV para su posterior envío.

A continuación, se muestra el código utilizado para esta sección:


In [ ]:
# Cargar el conjunto de test
test_df = pd.read_csv('test.csv')
print("Conjunto de test cargado exitosamente.")

# Limpiar el texto del conjunto de test
test_df['clean_text'] = test_df['text'].apply(clean_text)
print("Texto limpiado en el conjunto de test.")

# Transformar el texto limpio a la representación de características
X_test = tfidf_vectorizer.transform(test_df['clean_text']).toarray()
print(f"Tamaño de X_test: {X_test.shape}")

# Hacer las predicciones
y_test_pred = best_model.predict(X_test)
print("Predicciones realizadas con éxito.")

# Preparar el archivo de envío
submission_df = pd.DataFrame({'id': test_df['id'], 'target': y_test_pred})
submission_df.to_csv('submission.csv', index=False)
print("Archivo de envío preparado: 'submission.csv'")


Conjunto de test cargado exitosamente.
Texto limpiado en el conjunto de test.
Tamaño de X_test: (3263, 5000)
Predicciones realizadas con éxito.
Archivo de envío preparado: 'submission.csv'
